1. Package requirements for thsi include: Pillow opencv-python keras tensorflow pandas numpy
2. Need to download yolov3.cfg from [here](https://github.com/qqwweee/keras-yolo3/blob/master/yolov3.cfg), and yolov3.weights from [here](https://pjreddie.com/darknet/yolo/)
3. Convert the weights to a keras model with the `convert.py` script downloaded from [here](https://github.com/qqwweee/keras-yolo3/blob/master/convert.py), and by running the following command:
```bash
python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5
```
4. Retrieve yolo class labels from [this repository](https://github.com/leggedrobotics/darknet_ros/blob/master/darknet_ros/config/yolov3.yaml), and save it as json config

Note: some imports of the `convert.py` script are broken, corrected version included in current repository

TO DO:
- Add the class label to the image
- Add other of the 80 class levels (not only driving related)
- Modify the drawing function to make it more efficient

In [1]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [2]:
with open('server/demoData/classes.json', 'r') as handle:
    class_dict = json.load(handle)
    class_dict = {int(key):val for key,val in class_dict.items()}

In [22]:
# Run the YOLO model to detect objects.
def load_network(config_path="yolov3.cfg", weights_path="yolov3.weights"):
        net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
        output_layer_names = net.getLayerNames()
        output_layer_names = [output_layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
        return net, output_layer_names

def yolo_v3(image, net, output_layer_names, confidence_threshold=0.8, overlap_threshold=0.6, class_dict=class_dict):
    # Run the YOLO neural net.
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(output_layer_names)
    print(layer_outputs)
    # Supress detections in case of too low confidence or too much overlap.
    boxes, confidences, class_IDs = [], [], []
    H, W = image.shape[:2]
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > confidence_threshold:
                box = detection[0:4] * np.array([W, H, W, H])
                centerX, centerY, width, height = box.astype("int")
                x, y = int(centerX - (width / 2)), int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_IDs.append(classID)
    print(boxes)
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, overlap_threshold)
    if len(indices) > 0:
        # loop over the indexes we are keeping
        for i in indices.flatten():
            label = class_dict.get(class_IDs[i], None)
            if label is None:
                continue
            # extract the bounding box coordinates
            x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
            cv2.rectangle(image, (x,y), (x+w,y+h),(0,255,0),2)
            label_conf = "{0:.0%}".format(confidences[i])
            text_label = f"{label}: {label_conf}"
            (text_w, text_h), _ = cv2.getTextSize(
                text_label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1
            )
            # Prints the text.    
            image = cv2.rectangle(image, (x, y - 20), (x + text_w, y), (0,255,0), -1)
            image = cv2.putText(image, text_label, (x, y - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
            return image
    



In [23]:
net, output_layer_name = load_network(config_path="server/demoData/yolov4-tiny.cfg", weights_path="server/demoData/yolov4-tiny.weights")

In [24]:
import pickle
import urllib
import io
from PIL import Image
with open("server/demoData/payload.pkl", "rb") as handle:
    url = pickle.load(handle)

In [25]:
with urllib.request.urlopen(url) as file:
    content = file.read()
nparr = np.frombuffer(content, np.uint8)
img = cv2.imdecode(nparr, flags=cv2.IMREAD_COLOR)
image = yolo_v3(img, net, output_layer_name)

(array([[0.06135021, 0.0514141 , 0.16222665, ..., 0.        , 0.        ,
        0.        ],
       [0.05640702, 0.0358826 , 0.2058261 , ..., 0.        , 0.        ,
        0.        ],
       [0.04281428, 0.02809551, 0.6480995 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9365355 , 0.9499279 , 0.20856318, ..., 0.        , 0.        ,
        0.        ],
       [0.9382085 , 0.9396104 , 0.26385367, ..., 0.        , 0.        ,
        0.        ],
       [0.9576482 , 0.9354702 , 0.57431793, ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.02665966, 0.01862338, 0.0605887 , ..., 0.        , 0.        ,
        0.        ],
       [0.0264577 , 0.03181205, 0.07697888, ..., 0.        , 0.        ,
        0.        ],
       [0.01927491, 0.02908181, 0.25845245, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.97184473, 0.9744586 , 0.05659508, ..., 0.        , 0.        ,
        0.        ],
       [0.9751301 

In [13]:
net

<dnn_Net 0000022FF531ED10>

In [8]:
img_str = cv2.imencode('.png', image)[1].tobytes()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:976: error: (-215:Assertion failed) !image.empty() in function 'cv::imencode'


In [9]:
cv2.imencode('.png', image)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:976: error: (-215:Assertion failed) !image.empty() in function 'cv::imencode'


In [ ]:
base64.encodebytes(img_str)

In [ ]:
net.dumpToFile("model.pb")

In [ ]:
mod = cv2.dnn.readNet("model.pb")

In [ ]:
net